In [ ]:
#|default_exp text_editor

# Text Editor

Implements functions for Anthropic's Text Editor Tool API, allowing Claude to view and edit files.

See the [Anthropic Text Editor Tool documentation](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/text-editor-tool) for details on the API commands (`view`, `create`, `insert`, `str_replace`, `undo_edit`) and usage. The `str_replace_editor` function acts as the required dispatcher.

In [ ]:
#| export
from pathlib import Path

In [ ]:
#| hide
from claudette.core import *
from toolslm.funccall import *

In [ ]:
#| exports
def view(path:str, view_range:tuple[int,int]=None, nums:bool=False):
    'View directory or file contents with optional line range and numbers'
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists(): return f'Error: File not found: {p}'
        if p.is_dir(): return f'Directory contents of {p}:\n' + '\n'.join([str(f) for f in p.glob('**/*')
                                                                           if not f.name.startswith('.')])
        
        lines = p.read_text().splitlines()
        s,e = 1,len(lines)
        if view_range:
            s,e = view_range
            if not (1 <= s <= len(lines)): return f'Error: Invalid start line {s}'
            if e != -1 and not (s <= e <= len(lines)): return f'Error: Invalid end line {e}'
            lines = lines[s-1:None if e==-1 else e]
            
        return '\n'.join([f'{i+s-1:6d} │ {l}' for i,l in enumerate(lines,1)] if nums else lines)
    except Exception as e: return f'Error viewing file: {str(e)}'

In [ ]:
print(view('_quarto.yml', (1,10), nums=True))

     1 │ project:
     2 │   type: website
     3 │   resources: 
     4 │     - "*.txt"
     5 │   preview:
     6 │     port: 3000
     7 │     browser: false
     8 │ 
     9 │ format:
    10 │   html:


In [ ]:
#| exports
def create(path: str, file_text: str, overwrite:bool=False) -> str:
    'Creates a new file with the given content at the specified path'
    try:
        p = Path(path)
        if p.exists():
            if not overwrite: return f'Error: File already exists: {p}'
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(file_text)
        return f'Created file {p} containing:\n{file_text}'
    except Exception as e: return f'Error creating file: {str(e)}'

In [ ]:
print(create('test.txt', 'Hello, world!'))
print(view('test.txt', nums=True))

Created file test.txt containing:
Hello, world!
     1 │ Hello, world!


In [ ]:
#| exports
def insert(path: str, insert_line: int, new_str: str) -> str:
    'Insert new_str at specified line number'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text().splitlines()
        if not (0 <= insert_line <= len(content)): return f'Error: Invalid line number {insert_line}'
            
        content.insert(insert_line, new_str)
        new_content = '\n'.join(content)
        p.write_text(new_content)
        return f'Inserted text at line {insert_line} in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error inserting text: {str(e)}'

In [ ]:
insert('test.txt', 0, 'Let\'s add a new line')
print(view('test.txt', nums=True))

     1 │ Let's add a new line
     2 │ Hello, world!


In [ ]:
#| exports
def str_replace(path: str, old_str: str, new_str: str) -> str:
    'Replace first occurrence of old_str with new_str in file'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text()
        count = content.count(old_str)
        
        if count == 0: return 'Error: Text not found in file'
        if count > 1: return f'Error: Multiple matches found ({count})'
            
        new_content = content.replace(old_str, new_str, 1)
        p.write_text(new_content)
        return f'Replaced text in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error replacing text: {str(e)}'

In [ ]:
str_replace('test.txt', 'new line', '')
print(view('test.txt', nums=True))

     1 │ Let's add a 
     2 │ Hello, world!


In [ ]:
#| exports
def str_replace_editor(**kwargs: dict[str, str]) -> str:
    'Dispatcher for Anthropic Text Editor tool commands: view, str_replace, create, insert, undo_edit.'
    cmd = kwargs.pop('command', '')
    if cmd == 'view': return view(**kwargs)
    elif cmd == 'str_replace': return str_replace(**kwargs) 
    elif cmd == 'create': return create(**kwargs)
    elif cmd == 'insert': return insert(**kwargs)
    elif cmd == 'undo_edit': return 'Undo edit is not supported yet.'
    else: return f'Unknown command: {cmd}'

In [ ]:
#| export
text_editor_conf = {'sonnet':     {'type': 'text_editor_20250124',
                                   'name': 'str_replace_editor'},
                    'sonnet-3-5': {'type': 'text_editor_20241022',
                                   'name': 'str_replace_editor'}}

In [ ]:
c = Chat(models[1], tools=[text_editor_conf['sonnet']], ns=mk_ns(str_replace_editor))
c.toolloop('Please explain what my _quarto.yml does. Use your tools')

Based on the content of your `_quarto.yml` file, here's an explanation of what it does:

### Overview
This is a configuration file for Quarto, which is a scientific and technical publishing system. Your configuration sets up a website project with specific formatting and layout options.

### Key Components:

1. **Project Configuration**:
   - `type: website` - Defines this as a website project (as opposed to a book or other format)
   - `resources: - "*.txt"` - Includes all .txt files as resources
   - `preview` settings configure the development server to run on port 3000 without automatically opening a browser

2. **Format Settings**:
   - `html` section configures how HTML output is generated:
     - Uses the "cosmo" theme
     - Applies custom styles from styles.css
     - Enables table of contents (`toc: true`)
     - Configures code blocks with background, left border (#31BAE9 blue color), and "arrow" highlighting style
     - Sets specific grid layout dimensions for sidebar, body, margins, and gutters
     - `keep-md: true` preserves Markdown files after rendering
   - `commonmark: default` enables CommonMark format support

3. **Website Configuration**:
   - Enables Twitter card and Open Graph metadata for better social media sharing
   - Adds GitHub issue creation functionality (`repo-actions: [issue]`)
   - Configures a navbar with primary background color and search functionality
   - Sets up a floating sidebar

4. **Metadata Files**:
   - Includes two external configuration files: `nbdev.yml` and `sidebar.yml`
   - These likely contain additional settings for nbdev integration and sidebar structure

This configuration creates a website with a professional appearance, optimized for displaying code (suggesting this might be a technical documentation site or blog), with good navigation through the sidebar and navbar, and social media integration.

<details>

- id: `msg_01AxwZBpwK24FTKk4Y93EZnU`
- content: `[{'citations': None, 'text': 'Based on the content of your `_quarto.yml` file, here\'s an explanation of what it does:\n\n### Overview\nThis is a configuration file for Quarto, which is a scientific and technical publishing system. Your configuration sets up a website project with specific formatting and layout options.\n\n### Key Components:\n\n1. **Project Configuration**:\n   - `type: website` - Defines this as a website project (as opposed to a book or other format)\n   - `resources: - "*.txt"` - Includes all .txt files as resources\n   - `preview` settings configure the development server to run on port 3000 without automatically opening a browser\n\n2. **Format Settings**:\n   - `html` section configures how HTML output is generated:\n     - Uses the "cosmo" theme\n     - Applies custom styles from styles.css\n     - Enables table of contents (`toc: true`)\n     - Configures code blocks with background, left border (#31BAE9 blue color), and "arrow" highlighting style\n     - Sets specific grid layout dimensions for sidebar, body, margins, and gutters\n     - `keep-md: true` preserves Markdown files after rendering\n   - `commonmark: default` enables CommonMark format support\n\n3. **Website Configuration**:\n   - Enables Twitter card and Open Graph metadata for better social media sharing\n   - Adds GitHub issue creation functionality (`repo-actions: [issue]`)\n   - Configures a navbar with primary background color and search functionality\n   - Sets up a floating sidebar\n\n4. **Metadata Files**:\n   - Includes two external configuration files: `nbdev.yml` and `sidebar.yml`\n   - These likely contain additional settings for nbdev integration and sidebar structure\n\nThis configuration creates a website with a professional appearance, optimized for displaying code (suggesting this might be a technical documentation site or blog), with good navigation through the sidebar and navbar, and social media integration.', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1374, 'output_tokens': 438}`
- steps: `[{'role': 'user', 'content': 'Please explain what my _quarto.yml does. Use your tools'}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I'll examine your `_quarto.yml` file to explain what it does. Let me first view the file.", 'type': 'text'}, {'id': 'toolu_017KZjcxBAuzuJsDuKaSjuC3', 'input': {'command': 'view', 'path': '_quarto.yml'}, 'name': 'str_replace_editor', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_017KZjcxBAuzuJsDuKaSjuC3', 'content': 'project:\n  type: website\n  resources: \n    - "*.txt"\n  preview:\n    port: 3000\n    browser: false\n\nformat:\n  html:\n    theme: cosmo\n    css: styles.css\n    toc: true\n    code-tools: true\n    code-block-bg: true\n    code-block-border-left: "#31BAE9"\n    highlight-style: arrow\n    grid:\n      sidebar-width: 180px\n      body-width: 1800px\n      margin-width: 150px\n      gutter-width: 1.0rem\n    keep-md: true\n  commonmark: default\n\nwebsite:\n  twitter-card: true\n  open-graph: true\n  repo-actions: [issue]\n  navbar:\n    background: primary\n    search: true\n  sidebar:\n    style: floating\n\nmetadata-files:\n  - nbdev.yml\n  - sidebar.yml\n'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': 'Based on the content of your `_quarto.yml` file, here\'s an explanation of what it does:\n\n### Overview\nThis is a configuration file for Quarto, which is a scientific and technical publishing system. Your configuration sets up a website project with specific formatting and layout options.\n\n### Key Components:\n\n1. **Project Configuration**:\n   - `type: website` - Defines this as a website project (as opposed to a book or other format)\n   - `resources: - "*.txt"` - Includes all .txt files as resources\n   - `preview` settings configure the development server to run on port 3000 without automatically opening a browser\n\n2. **Format Settings**:\n   - `html` section configures how HTML output is generated:\n     - Uses the "cosmo" theme\n     - Applies custom styles from styles.css\n     - Enables table of contents (`toc: true`)\n     - Configures code blocks with background, left border (#31BAE9 blue color), and "arrow" highlighting style\n     - Sets specific grid layout dimensions for sidebar, body, margins, and gutters\n     - `keep-md: true` preserves Markdown files after rendering\n   - `commonmark: default` enables CommonMark format support\n\n3. **Website Configuration**:\n   - Enables Twitter card and Open Graph metadata for better social media sharing\n   - Adds GitHub issue creation functionality (`repo-actions: [issue]`)\n   - Configures a navbar with primary background color and search functionality\n   - Sets up a floating sidebar\n\n4. **Metadata Files**:\n   - Includes two external configuration files: `nbdev.yml` and `sidebar.yml`\n   - These likely contain additional settings for nbdev integration and sidebar structure\n\nThis configuration creates a website with a professional appearance, optimized for displaying code (suggesting this might be a technical documentation site or blog), with good navigation through the sidebar and navbar, and social media integration.', 'type': 'text'}]}]`

</details>